# LLM Zoomcamp 2024

## Module 1: Introduction to LLMs and RAG

## Homework

In this homework, we will learn about and work on search using Elasticsearch.

Click [here](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md) to see the original homework document on the course's GitHub page.

In [52]:
!pip install requests tiktoken

In [1]:
import requests
import os
from dotenv import load_dotenv
import json
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

load_dotenv()

/home/codespace/miniconda3/envs/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:


```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [2]:
!curl localhost:9200

{
  "name" : "a83d84d58550",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "_lJb38Z9T1OmblawdwH4qQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [3]:
cluster_output = !curl -s localhost:9200  # Use -s to silence curl progress
cluster_json = ''.join(cluster_output)  # Convert SList to string

cluster_info = json.loads(cluster_json)

build_hash = cluster_info["version"]["build_hash"]
print("version.build_hash value: ", build_hash)

version.build_hash value:  42f05b9372a9a4a470db3b52817899b99a76ee73


In [4]:
# url = "http://localhost:9200"

# response = requests.get(url)

# if response.status_code == 200:
#     cluster_info = response.json()
    
#     build_hash = cluster_info["version"]["build_hash"]
    
#     print("version.build_hash value: ", build_hash)
# else:
#     print("Failed to retrieve version.build_hash value. Status code:", response.status_code)
    

### Getting the data

Now, let's get the FAQ data by running the following code:


In [5]:
docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]
    
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
len(documents)

948

### Q2. Indexing the data

Now, we will index the data making the `course` field a keyword and the rest a text.

In [8]:
es_client = Elasticsearch("http://localhost:9200")

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  1%|          | 9/948 [00:00<00:23, 39.13it/s]

100%|██████████| 948/948 [00:23<00:00, 40.96it/s]


We use the `index()` function to add our data to Elasticsearch.

### Q3. Searching

Now, let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

We will use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

We will find the score for the top ranking result looking at the `_score` field.



In [11]:
query = "How do I execute a command in a running docker container?"

In [12]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }  
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

top_score = response["hits"]["hits"][0]["_score"]

print("Score for the top ranking result:", top_score)

Score for the top ranking result: 84.050095


### Q4. Filtering

Now, let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?


In [13]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    results_docs = []

    for hit in response["hits"]["hits"]:
        results_docs.append(hit["_source"])
 
    return results_docs

In [15]:
search_results = elastic_search(query)
third_question = search_results[2]["question"]
print("Third question:", third_question)

Third question: How do I copy files from a different folder into docker container’s working directory?


### Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:


```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
            
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [28]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
print(len(prompt))

1652


### Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization.

Let's calculate the number of tokens in our query:




In [29]:
encoding = tiktoken.encoding_for_model("gpt-4o")

Use the `encode` function. How many tokens does our prompt have?

In [30]:
prompt_tokens = encoding.encode(prompt)
print("Number of tokens in our prompt:", len(prompt_tokens))

Number of tokens in our prompt: 359


Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

In [31]:
decoded_tokens = [encoding.decode_single_token_bytes(token).decode() for token in prompt_tokens]
decoded_string = ''.join(decoded_tokens)
decoded_string

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do I execute a command in a running docker container?\n\n    CONTEXT:\n    section: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nsection: 5. Deploying Machine Learning Models\nquestion: How do I copy files from my local machine to docker container?\nanswer: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into 

### Generating the answer

Now, let's send the prompt to OpenAI.


In [32]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

In [42]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. Here’s how you can do it:

1. First, find the container ID of the running container by using:
   ```bash
   docker ps
   ```
2. Then execute a command in the specific container using:
   ```bash
   docker exec -it <container-id> <command>
   ```

For example, to start an interactive bash shell in the container, you can run:
```bash
docker exec -it <container-id> bash
```

This allows you to interact with the running container directly.
